In [2]:
import sympy as sm
import numpy as np
import matplotlib.pyplot as plt

In [86]:
x= sm.symbols('x y')

In [89]:
a = sm.integrate((y-x)/(1-y))

TypeError: unsupported operand type(s) for -: 'Symbol' and 'tuple'